This notebook visualizes some of the content of the dataset

[Potoniec, Jedrzej; Wiśniewski, Dawid ; Ławrynowicz, Agnieszka; Keet, Maria (2019), “Ontology Competency Questions to SPARQL-OWL Queries Translations”, Mendeley Data, V1, doi: 10.17632/pp6hmfxgfg.1](https://data.mendeley.com/datasets/pp6hmfxgfg/1)


The dataset is licenced under **CC BY NC 3.0** and consists of 10 ontologies and 131 SPARQL-queries which are formalizations of competency questions. 

There is also a interesting [blog post](https://keet.wordpress.com/2020/01/09/a-set-of-competency-questions-and-sparql-owl-queries-with-analysis/) by M. Keet and a written publication:

[*Potoniec, J., Wisniewski, D., Lawrynowicz, A., Keet, C.M.: **Dataset of Ontology Competency Questions to SPARQL-OWL Queries Translations.** Data in Brief, 2020*](https://doi.org/10.1016/j.dib.2019.105098).



This notebook uses [Owlready2]() to interact with the ontologies, see also:

[*Lamy, JB: Owlready: **Ontology-oriented programming in Python with automatic classification and high level constructs for biomedical ontologies**. Artificial Intelligence In Medicine 2017;80:11-28*](http://www.lesfleursdunormal.fr/_downloads/article_owlready_aim_2017.pdf)

[*Lamy, JB: **Ontologies with Python**, Apress, 2020*](https://www.apress.com/fr/book/9781484265512)


In [1]:
# enable magic comments for more understandible output (this can be savely removed)
%load_ext ipydex.displaytools


x = 3 ##:

In [2]:
import owlready2 as owl2

## Ontology 1: African Wildlife Ontology

In [3]:
fpath = "Ontologies/AfricanWildlifeOntology1.owl"
onto = owl2.get_ontology(fpath).load()
g = owl2.default_world.as_rdflib_graph()

In [4]:
rq = """
# awo_1.rq
# Which animal eats which other animal?

prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

select distinct *
where
{
    ?eats rdfs:subClassOf :animal, [
        a owl:Restriction ;
        owl:onProperty :eats;
        owl:someValuesFrom ?eaten
    ] .
    ?eaten rdfs:subClassOf :animal .
    filter(?eats != owl:Nothing)
}
"""
list(g.query_owlready(rq))

[[AfricanWildlifeOntology1.Impala, AfricanWildlifeOntology1.lion]]

Note: Question 2 uses a variable `$PPx1$` in the source. This is not directly suported by pythons rdflib. Therefore we generate a list of all animals before and then run question 2 for each of these.

In [5]:
rq = """
# preparation for awo_2.rq
# Is [this animal] a herbivore?


prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

select distinct ?X
where
{
    ?X rdfs:subClassOf :animal .
}
"""

nested_list = list(g.query_owlready(rq)) ##:
animal_list = [e[0].name for  e in nested_list] ##:

nested_list := [[AfricanWildlifeOntology1.herbivore],
 [AfricanWildlifeOntology1.Impala],
 [AfricanWildlifeOntology1.Omnivore],
 [AfricanWildlifeOntology1.carnivore],
 [AfricanWildlifeOntology1.RockDassie],
 [AfricanWildlifeOntology1.Warthog],
 [AfricanWildlifeOntology1.giraffe],
 [AfricanWildlifeOntology1.lion]]

---

animal_list := ['herbivore',
 'Impala',
 'Omnivore',
 'carnivore',
 'RockDassie',
 'Warthog',
 'giraffe',
 'lion']

---

In [8]:
rq = """
# awo_2.rq
# Is [this animal] a herbivore?


prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

ask where
{
$PPx1$ rdfs:subClassOf :animal, :herbivore .
}

"""

for animal in animal_list:
    res = list(g.query_owlready(rq.replace("$PPx1$", f":{animal}")))
    print(f"{animal}: {res}")



herbivore: [False]
Impala: [False]
Omnivore: [False]
carnivore: [False]
RockDassie: [False]
Warthog: [False]
giraffe: [False]
lion: [False]


The results looks strange: `giraffe` should be recognized as subclass of `herbivore`. (Note that the ontology does not contain any information on the eating disposition of `Impala`).

Solution: running the reasoner:

In [9]:
%time owl2.sync_reasoner(debug=0)

CPU times: user 8.84 ms, sys: 3.82 ms, total: 12.7 ms
Wall time: 1.98 s


In [10]:
for animal in animal_list:
    res = list(g.query_owlready(rq.replace("$PPx1$", f":{animal}")))
    print(f"{animal}: {res}")

herbivore: [False]
Impala: [False]
Omnivore: [False]
carnivore: [False]
RockDassie: [False]
Warthog: [False]
giraffe: [True]
lion: [False]


Note: The numbering of the files in the dataset is not consecutive. In particular, the following files are present:

```
awo_1.rq
awo_2.rq
awo_4.rq
awo_6.rq
awo_7.rq
awo_8.rq
awo_14.rq
```

In [11]:
rq = """
# awo_4.rq
# Does a lion eat plants or plant parts?


prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

ask
where
{
    :lion rdfs:subClassOf [
        a owl:Restriction ;
        owl:onProperty :eats ;
        owl:someValuesFrom [
            owl:unionOf (:plant :PlantParts)
        ]
    ] .
}
"""

list(g.query_owlready(rq))

[False]

In [12]:
rq = """
# awo_6.rq
# Which plants eat animals?

prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

select distinct ?eats
where
{
    ?eats rdfs:subClassOf :plant, [
            a owl:Restriction ;
            owl:onProperty :eats;
            owl:someValuesFrom :animal
        ] .
    filter(?eats != owl:Nothing)
}
"""
list(g.query_owlready(rq))

[[AfricanWildlifeOntology1.CarnivorousPlant]]

Note: awo_t.rq again contains a variable (`$PPx1$`). We use the same approach as for awo_2.rq

In [13]:
rq = """
# awo_7.rq
# Which animals eat [these animals]?


prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

select distinct *
where
{
    ?eats rdfs:subClassOf :animal, [
        a owl:Restriction ;
        owl:onProperty :eats;
        owl:someValuesFrom $PPx1$
    ] .
    $PPx1$ rdfs:subClassOf :animal .
    filter(?eats != owl:Nothing)
}
"""

for animal in animal_list:
    res = list(g.query_owlready(rq.replace("$PPx1$", f":{animal}")))
    print(f"{animal}: {res}")

herbivore: []
Impala: [[AfricanWildlifeOntology1.lion]]
Omnivore: []
carnivore: []
RockDassie: []
Warthog: []
giraffe: []
lion: []


In [14]:
rq = """
# awo_8.rq
# Which animals are the predators of [these animals]?


prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>
select distinct *
where
{
    ?eats rdfs:subClassOf :animal, [
        a owl:Restriction ;
        owl:onProperty :eats;
        owl:someValuesFrom $PPx1$
        ] .
    $PPx1$ rdfs:subClassOf :animal .
    filter(?eats != owl:Nothing)
}
"""

for animal in animal_list:
    res = list(g.query_owlready(rq.replace("$PPx1$", f":{animal}")))
    print(f"{animal}: {res}")
    

herbivore: []
Impala: [[AfricanWildlifeOntology1.lion]]
Omnivore: []
carnivore: []
RockDassie: []
Warthog: []
giraffe: []
lion: []


In [15]:
rq = """
# Are there animals that are carnivore but still eat some plants or parts of plants?


prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix : <http://www.meteck.org/teaching/ontologies/AfricanWildlifeOntology1.owl#>

ask
where
{
    filter not exists {
    :carnivore owl:disjointWith [
            a owl:Restriction ;
            owl:onProperty :eats ;
                owl:someValuesFrom [
                owl:unionOf (:plant :PlantParts)
            ]
        ] .
    }
}

"""
list(g.query_owlready(rq))

[True]